In [1]:
import pandas as pd
import numpy as num
import statsmodels.api as sm
from datetime import timedelta

C:\Users\Carlos Bonilla\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
#Upload csv and excel datasets
Savefolder = "C:\\Users\\Carlos Bonilla\\Google Drive\\Wharton\\Clubs\\Wharton Analytic Fellows\\"
Fitting_Data_All=pd.read_excel(Savefolder+"Fittting Data (World Bank Global Economic Indicators) v4.xlsx", sheet_name="Data")
ECRCountriesFitted = pd.read_excel(Savefolder+"ECRCountriesFitted.xlsx", sheet_name=None)

In [98]:
#Prepare ECR General score data for analysis
ECR=pd.read_excel(Savefolder+"ECR Testing Data.xlsx", sheet_name='ECR_SubScores')

#ECR.groupby(pd.TimeGrouper(freq='M'),'country').mean()
#ECR.groupby('country',pd.TimeGrouper(freq='M')).mean()
ECRA = ECR.pivot(index='date', columns='country', values='General_Score')
ECRA = ECRA.resample('M').mean()
ECRA['date'] = pd.to_datetime(ECRA.index)

In [162]:
#Prepare World Bank monthly data for analysis
###MANUALLY changed Egypt, Russia and Korea South name###
Fitting_Data_All=pd.read_excel(Savefolder+"Fittting Data (World Bank Global Economic Indicators) v4.xlsx", sheet_name="Data")

,Country,Country Code,Series,Series Code,1989-01-31 00:00:00,1989-02-28 00:00:00,1989-03-31 00:00:00,1989-04-30 00:00:00,1989-05-31 00:00:00,1989-06-30 00:00:00,...,2017-02-28 00:00:00,2017-03-31 00:00:00,2017-04-30 00:00:00,2017-05-31 00:00:00,2017-06-30 00:00:00,2017-07-31 00:00:00,2017-08-31 00:00:00,2017-09-30 00:00:00,2017-10-31 00:00:00,2017-11-30 00:00:00
0,Afghanistan,AFG,"CPI Price, % y-o-y, median weighted, seas. adj.,",CPTOTSAXMZGY,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,AFG,"Exchange rate, new LCU per USD extended backwa...",DPANUSSPB,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,"Exports Merchandise, Customs, constant US$, mi...",DXGSRMRCHSAKD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,AFG,"GDP,constant 2010 US$,millions,seas. adj.,",NYGDPMKTPSAKD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,AFG,"Imports Merchandise, Customs, constant US$, mi...",DMGSRMRCHSAKD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Afghanistan,AFG,"Imports Merchandise, Customs, Price, US$, seas...",DMGSRMRCHSAXD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Afghanistan,AFG,"Industrial Production, constant US$, seas. adj.,,",IPTOTSAKD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Afghanistan,AFG,"Months Import Cover of Foreign Reserves,,,,",IMPCOV,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Afghanistan,AFG,"Real Effective Exchange Rate,,,,",REER,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Afghanistan,AFG,"Retail Sales Volume,Index,,,",RETSALESSA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [176]:
import statsmodels.api as sm

model = linear_model.LinearRegression()

dateAndCountryDict = {}

for theCountry in ECRCountriesFitted['Country'].unique():
    
    singleCountryDict = {}    
    
    for x in range(0, len( pd.to_datetime(ECRA.index))-12):
        try:
            print(theCountry)
            tempDf = Fitting_Data_All.loc[Fitting_Data_All['Country'] == theCountry]
            del tempDf['Country Code']
            del tempDf['Series']
            del tempDf['Country']
            tempDf = pd.melt(tempDf, id_vars=["Series Code"],var_name="date")
            tempDf = tempDf.pivot(index='date', columns='Series Code', values='value')
            tempDf['date'] = tempDf.index
            tempDf = pd.merge(ECRA[[theCountry,'date']],tempDf, how='left', on='date',left_index=False, 
                  right_index=False, sort=False, copy=True, indicator=False)
            tempDf = tempDf.dropna(axis=1, how='any')
            subDf = tempDf[x:x+28]
            result = sm.OLS(subDf[theCountry],subDf.iloc[:,3:20]).fit()
        except ValueError:
            pass     
        singleCountryDict[x] = result
    dateAndCountryDict[theCountry] = singleCountryDict

Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Brazil
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Bulgaria
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Chile
Ch

Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey
Turkey


In [177]:
from sklearn.metrics import mean_squared_error, r2_score

#print('Coefficients: \n', regr.coef_)
#print (dateAndCountryDict['Turkey'][0].intercept_, dateAndCountryDict['Turkey'][0].coef_)
#get_params(dateAndCountryDict['Turkey'][0])


print(dateAndCountryDict['Turkey'][0].summary())

                            OLS Regression Results                            
Dep. Variable:                 Turkey   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.917e+04
Date:                Mon, 20 Nov 2017   Prob (F-statistic):           1.60e-32
Time:                        11:45:52   Log-Likelihood:                -20.595
No. Observations:                  28   AIC:                             63.19
Df Residuals:                      17   BIC:                             77.84
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
DMGSRMRCHSAXD    19.4154      8.484      2.289

Sources and citations:

http://marcharper.codes/2016-06-14/Linear+Regression+with+Statsmodels+and+Scikit-Learn.html

Sherouse, Oliver (2014). Wbdata. Arlington, VA. Available from http://github.com/OliverSherouse/wbdata

https://blogs.worldbank.org/opendata/accessing-world-bank-data-apis-python-r-ruby-stata